In [2]:
import copy
import uuid
import os
import botocore.exceptions
import random
import pandas as pd
import awswrangler as wr
import boto3
import json

In [3]:
glue = boto3.client('glue')
s3 = boto3.client('s3')
client = boto3.client("bedrock-runtime", region_name="us-west-2")

In [4]:
def listar_glue_jobs():
    paginator = glue.get_paginator('get_jobs')
    all_jobs = []

    for page in paginator.paginate():
        for job in page['Jobs']:
            job_info = {
                'name': job['Name'],
                'script_location': job['Command']['ScriptLocation'],
            }
            all_jobs.append(job_info)

    return all_jobs

In [5]:
jobs = listar_glue_jobs()

In [6]:
def clonar_e_alterar_glue_job(nome_origem, nome_destino, novo_codigo, pasta="scripts-novos"):
    job_response = glue.get_job(JobName=nome_origem)
    job_def = job_response['Job']
    
    script_location = job_def['Command']['ScriptLocation']
    if not script_location.startswith("s3://"):
        raise ValueError("ScriptLocation inválido.")
    
    _, _, bucket, *chave = script_location.split("/")
    chave_original = "/".join(chave)
    
    nome_arquivo = os.path.basename(chave_original)
    
    pasta_base = os.path.dirname(chave_original).rstrip("/")
    novo_arquivo = f"{nome_destino}"
    novo_caminho = f"{pasta_base}/{pasta}/{novo_arquivo}".replace("//", "/")[1:] + '.py'

    s3.put_object(
        Bucket=bucket,
        Key=novo_caminho,
        Body=novo_codigo.encode("utf-8")
    )
    
    campos_remover = [
        'Name', 'CreatedOn', 'LastModifiedOn', 'ExecutionProperty', 
        'MaxRetries', 'AllocatedCapacity', 'GlueVersion', 'Tags',
        'ExecutionClass', 'Timeout', 'JobBookmarkOption'
    ]
    for campo in campos_remover:
        job_def.pop(campo, None)

    job_def['Command']['ScriptLocation'] = 's3://' + f"{bucket}/{novo_caminho}".replace("//", "/")
    
    try:
        glue.create_job(Name=nome_destino, **job_def)
        acao = "criado"
    except glue.exceptions.IdempotentParameterMismatchException:
        glue.update_job(JobName=nome_destino, JobUpdate=job_def)
        acao = "atualizado"
    
    return {
        "novo_job": nome_destino,
        "script_s3": job_def['Command']['ScriptLocation'],
        "acao": acao
    }

In [7]:
def listar_subpastas(bucket):
    resposta = s3.list_objects_v2(Bucket=bucket, Delimiter='/')
    subpastas = [
        pasta['Prefix'].rstrip('/')
        for pasta in resposta.get('CommonPrefixes', [])
        if pasta['Prefix'].rstrip('/') != 'athena'
    ]
    return subpastas

In [8]:
def gerar_codigo_exemplo_com_logging(result):
    return random.choice(list(result.keys()))

In [9]:
def criar_jobs_para_subpastas(subpastas, bucket, job_template, numero_servicos=20):
    resultados = []
    with open("out.json", "r") as fp:
        result = json.load(fp)
        for subpasta in subpastas:
            for i in range(1, numero_servicos + 1):
                nome_novo_job = f"{subpasta}_servico{i}"
                novo_codigo = gerar_codigo_exemplo_com_logging(result)
                resultado = clonar_e_alterar_glue_job(
                    nome_origem=job_template,
                    nome_destino=nome_novo_job,
                    novo_codigo=novo_codigo,
                    pasta=subpasta
                )
                print(f"✅ Criado: {resultado['novo_job']} em {subpasta}")
                resultados.append(resultado)
    return resultados

In [10]:
def ler_scripts_dos_jobs(lista_jobs):
    resultados = []

    for job in lista_jobs:
        nome = job['name']
        caminho = job['script_location']

        caminho = caminho.replace("s3://", "")
        partes = caminho.split("/", 1)
        bucket = partes[0]
        chave = partes[1]

        resposta = s3.get_object(Bucket=bucket, Key=chave)
        conteudo = resposta['Body'].read().decode('utf-8')

        if nome == "template":
            continue

        if "_" in nome:
            area, nome_servico = nome.split("_", 1)
        else:
            area = "desconhecido"
            nome_servico = nome

        resultados.append({
            "area": area,
            "nome_servico": nome_servico,
            "codigo": conteudo
        })

    return resultados

In [11]:
bucket = 'costs-data-123'
subpastas = listar_subpastas(bucket)
print("Subpastas:", subpastas)

Subpastas: ['cartoes', 'financas', 'finops', 'pix', 'ti']


In [12]:
job_template = "template"
resultados = criar_jobs_para_subpastas(subpastas, bucket, job_template)

✅ Criado: cartoes_servico1 em cartoes
✅ Criado: cartoes_servico2 em cartoes
✅ Criado: cartoes_servico3 em cartoes
✅ Criado: cartoes_servico4 em cartoes
✅ Criado: cartoes_servico5 em cartoes
✅ Criado: cartoes_servico6 em cartoes
✅ Criado: cartoes_servico7 em cartoes
✅ Criado: cartoes_servico8 em cartoes
✅ Criado: cartoes_servico9 em cartoes
✅ Criado: cartoes_servico10 em cartoes
✅ Criado: cartoes_servico11 em cartoes
✅ Criado: cartoes_servico12 em cartoes
✅ Criado: cartoes_servico13 em cartoes
✅ Criado: cartoes_servico14 em cartoes
✅ Criado: cartoes_servico15 em cartoes
✅ Criado: cartoes_servico16 em cartoes
✅ Criado: cartoes_servico17 em cartoes
✅ Criado: cartoes_servico18 em cartoes
✅ Criado: cartoes_servico19 em cartoes
✅ Criado: cartoes_servico20 em cartoes
✅ Criado: financas_servico1 em financas
✅ Criado: financas_servico2 em financas
✅ Criado: financas_servico3 em financas
✅ Criado: financas_servico4 em financas
✅ Criado: financas_servico5 em financas
✅ Criado: financas_servico6 e

In [13]:
servicos_existentes = ler_scripts_dos_jobs(jobs)

In [14]:
def analisar_codigo_por_ia(result, codigo):
    return result[codigo]

In [15]:
def analisar_servicos_com_ia(servicos_existentes):
    resultados = []
    with open("out.json", "r") as fp:
        result = json.load(fp)
        for servico in servicos_existentes:
            area = servico['area']
            nome_servico = servico['nome_servico']
            codigo = servico['codigo']
            analise = analisar_codigo_por_ia(result, codigo)
            
            script_name = f"{nome_servico}_temp.py"
            new_key = f"{area}/{script_name}"
            bucket = 'costs-data-123'
            
            s3.put_object(
                Bucket=bucket,
                Key=new_key,
                Body=analise["codigo_sugestao"].encode("utf-8")
            )
            
            new_script_location = f"s3://{bucket}/{new_key}"
            
            resultado = {
                "area": area,
                "nome_servico": nome_servico,
                "codigo": codigo,
                "problema_log": analise["problema_log"],
                "quantidade_problemas_log": int(analise["quantidade_problemas_log"]),
                "codigo_sugestao": analise["codigo_sugestao"],
                "new_script_location": new_script_location
            }
            resultados.append(resultado)
    return resultados

In [16]:
servicos_analisados = analisar_servicos_com_ia(servicos_existentes)

for s in servicos_analisados:
    print(f"{s['area']}/{s['nome_servico']} -> problema_log: {s['problema_log']} ({s['quantidade_problemas_log']})")
    print(f"Nova versão salva em: {s['new_script_location']}")
    print("--------")


cartoes/servico1 -> problema_log: False (5)
Nova versão salva em: s3://costs-data-123/cartoes/servico1_temp.py
--------
cartoes/servico10 -> problema_log: False (2)
Nova versão salva em: s3://costs-data-123/cartoes/servico10_temp.py
--------
cartoes/servico11 -> problema_log: False (4)
Nova versão salva em: s3://costs-data-123/cartoes/servico11_temp.py
--------
cartoes/servico12 -> problema_log: False (6)
Nova versão salva em: s3://costs-data-123/cartoes/servico12_temp.py
--------
cartoes/servico13 -> problema_log: True (6)
Nova versão salva em: s3://costs-data-123/cartoes/servico13_temp.py
--------
cartoes/servico14 -> problema_log: False (3)
Nova versão salva em: s3://costs-data-123/cartoes/servico14_temp.py
--------
cartoes/servico15 -> problema_log: False (6)
Nova versão salva em: s3://costs-data-123/cartoes/servico15_temp.py
--------
cartoes/servico16 -> problema_log: True (6)
Nova versão salva em: s3://costs-data-123/cartoes/servico16_temp.py
--------
cartoes/servico17 -> problem

In [17]:
len(servicos_analisados)

100

In [18]:
def salvar_tabela_analitica_athena(servicos_analizados, bucket, database, nome_tabela="tb_sor_servicos_analitico"):
    df = pd.DataFrame(servicos_analizados)
    indicadores = ['problema_log', 'quantidade_problemas_log']
    df_melted = df.melt(id_vars=['area', 'nome_servico'], 
                        value_vars=indicadores, 
                        var_name='indicador', 
                        value_name='valor')
    df_melted['valor'] = df_melted['valor'].apply(lambda x: float(x) if isinstance(x, (int, float)) else 1.0 if x is True else 0.0)
    
    caminho_s3 = f"s3://{bucket}/athena/servicos_analitico/tb_sor_servicos_analitico"
    wr.s3.to_parquet(
        df=df_melted,
        path=caminho_s3,
        dataset=True,
        database=database,
        table=nome_tabela,
        mode="overwrite"
    )
    print(f"✅ Tabela '{nome_tabela}' criada no Athena (DB: {database}) com {len(df_melted)} registros.")

In [19]:
salvar_tabela_analitica_athena(
    servicos_analizados=servicos_analisados,
    bucket='costs-data-123',
    database='workspace'
)

2025-06-08 18:02:22,065	WARNING services.py:2070 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 8312041472 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-06-08 18:02:22,214	INFO worker.py:1852 -- Started a local Ray instance.


✅ Tabela 'tb_sor_servicos_analitico' criada no Athena (DB: workspace) com 200 registros.


In [20]:
query = f"SELECT * FROM tb_sor_servicos_analitico"
    
df = wr.athena.read_sql_query(
    sql=query,
    database='workspace'
)

print("📊 Dados lidos da tabela:")
display(df)

📊 Dados lidos da tabela:


,area,nome_servico,indicador,valor
0,cartoes,servico1,problema_log,0.0
1,cartoes,servico10,problema_log,0.0
2,cartoes,servico11,problema_log,0.0
3,cartoes,servico12,problema_log,0.0
4,cartoes,servico13,problema_log,1.0
...,...,...,...,...
195,ti,servico5,quantidade_problemas_log,3.0
196,ti,servico6,quantidade_problemas_log,3.0
197,ti,servico7,quantidade_problemas_log,3.0
198,ti,servico8,quantidade_problemas_log,6.0
